## Parse input

In [1]:
from typing import Optional, Any, List

In [2]:
from PIL import Image
import os
from tqdm import tqdm

def split_and_save_image(filename: str, input_image_path: str, output_folder_path: str, save_to_tmp: Optional[bool] = False) -> str:
    """
    Разделяет изображение на части и сохраняет их в указанную директорию.

    Аргументы:
    - filename: Имя файла изображения.
    - input_image_path: Путь к директории с исходным изображением.
    - output_folder_path: Путь к директории, куда будут сохранены части изображения.
    - save_to_tmp: Флаг, указывающий на сохранение во временную папку. По умолчанию False.

    Возвращает:
    - Имя папки, в которую были сохранены части изображения.
    """
    # Open the input image
    with Image.open(os.path.join(input_image_path, filename)) as img:

        # Get the filename and the extension
        extension = os.path.basename(filename).split('.')[-1]

        if save_to_tmp:
            foldername = "tmp"
        else:
            foldername = filename.split('.')[0]

        # Define the size of each individual image, assuming they are of equal height
        # and the last image occupies the whole width of the original image
        width, height = img.size
        single_image_height = height // 2  # Divide by 2 because there are 2 rows
        single_image_width = width // 5  # Divide by 5 for the images in the first row
        
        # Create the output directory
        output_directory = os.path.join(output_folder_path, foldername)
        os.makedirs(output_directory, exist_ok=True)
        
        # Split the images and save them
        for i in range(5):  # For the first row
            left = i * single_image_width
            right = (i + 1) * single_image_width
            box = (left, 0, right, single_image_height)
            part_img = img.crop(box)
            part_img.save(os.path.join(output_directory, f'x_{i+1}.{extension}'))
        
        # Save the last image
        box = (0, single_image_height, width*(2/15), 2 * single_image_height)
        part_img = img.crop(box)
        part_img.save(os.path.join(output_directory, f'y.{extension}'))

        return foldername

## Crop squares

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from copy import deepcopy
from PIL import Image, ImageDraw



def open_image_and_visualize(image_path: str, visualize: Optional[bool] = False) -> np.ndarray:
    """
    Открывает изображение по указанному пути и при необходимости визуализирует его.

    Аргументы:
    - image_path: Путь к изображению.
    - visualize: Флаг, определяющий необходимость визуализации. По умолчанию False.

    Возвращает:
    - Массив изображения в формате numpy.ndarray.
    """
    im = cv2.imread(image_path)
    if visualize:
        plt.imshow(im)
        plt.colorbar()
        plt.show()
    return im


def make_scalar_product_mask(im: np.ndarray, visualize: Optional[bool] = False) -> np.ndarray:
    """
    Создает маску изображения на основе скалярного произведения и при необходимости визуализирует её.

    Аргументы:
    - im: Исходное изображение в формате numpy.ndarray.
    - visualize: Флаг, определяющий необходимость визуализации. По умолчанию False.

    Возвращает:
    - Маску изображения в формате numpy.ndarray.
    """
    image = im/255.
    unit_vector = np.array([1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3)])
    unit_image = image/np.sqrt(np.tile(np.sum(image*image, axis=2), (3,1,1)).transpose(1,2,0))
    new_im = np.sum(unit_image*unit_vector, axis=2) * 255
    if visualize:
        plt.imshow(new_im)
        plt.colorbar()
        plt.show()
    return new_im


def make_binar(im: np.ndarray, threshold: Optional[int] = 240, visualize: Optional[bool] = False) -> np.ndarray:
    """
    Бинаризует изображение по заданному порогу и при необходимости визуализирует его.

    Аргументы:
    - im: Исходное изображение в формате numpy.ndarray.
    - threshold: Порог для бинаризации. По умолчанию 240.
    - visualize: Флаг, определяющий необходимость визуализации. По умолчанию False.

    Возвращает:
    - Бинаризованное изображение в формате numpy.ndarray.
    """
    img = deepcopy(im)
    img[img!=img] = 255
    img[img<threshold] = 0
    img[img>=threshold] = 255
    if visualize:
        plt.imshow(img, cmap="gray")
        plt.colorbar()
        plt.show()
    return img


def pad_borders(im: np.ndarray, pad_width: Optional[int] = 3, visualize: Optional[bool] = False) -> np.ndarray:
    """
    Добавляет рамку вокруг изображения и при необходимости визуализирует его.

    Аргументы:
    - im: Исходное изображение в формате numpy.ndarray.
    - pad_width: Ширина рамки. По умолчанию 3.
    - visualize: Флаг, определяющий необходимость визуализации. По умолчанию False.

    Возвращает:
    - Изображение с рамкой в формате numpy.ndarray.
    """
    im = np.pad(im, 10, 'linear_ramp', end_values=255)
    if visualize:
        plt.imshow(im, cmap='gray')
        plt.colorbar()
        plt.show()
    return im


def smooth_circle_borders(im: np.ndarray, visualize: Optional[bool] = False) -> np.ndarray:
    """
    Сглаживает границы кругов на изображении и при необходимости визуализирует результат.

    Аргументы:
    - im: Исходное изображение в формате numpy.ndarray.
    - visualize: Флаг, определяющий необходимость визуализации. По умолчанию False.

    Возвращает:
    - Изображение с сглаженными границами в формате numpy.ndarray.
    """    
    kernel_size = 3
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    im = cv2.morphologyEx(im, cv2.MORPH_OPEN, kernel, iterations=2)
    im = cv2.morphologyEx(im, cv2.MORPH_CLOSE, kernel, iterations=1)
    if visualize:
        plt.imshow(im, cmap='gray')
        plt.colorbar()
        plt.show()
    return im


def detect_circles(im: np.ndarray, visualize: Optional[bool] = False) -> list:
    """
    Обнаруживает круги на изображении и при необходимости визуализирует результат.

    Аргументы:
    - im: Исходное изображение в формате numpy.ndarray.
    - visualize: Флаг, определяющий необходимость визуализации. По умолчанию False.

    Возвращает:
    - Список центров обнаруженных кругов.
    """
    im = np.uint8(im)
    radius_tolerance = 5
    min_radius = 20 - radius_tolerance
    max_radius = 20 + radius_tolerance

    dp = 1.5  # Обратное отношение разрешения накопителя квадратов к разрешению изображения
    minDist = 25  # Минимальное расстояние между центрами обнаруженных кругов
    param1 = 40  # Более высокий порог из двух, передаваемых в детектор краев Canny
    param2 = 18  # Порог аккумулятора для центров кругов на этапе обнаружения

    # Используем HoughCircles для обнаружения кругов
    detected_circles = cv2.HoughCircles(
        im, 
        cv2.HOUGH_GRADIENT, 
        dp, 
        minDist, 
        param1=param1, 
        param2=param2, 
        minRadius=min_radius, 
        maxRadius=max_radius
    )

    # Преобразуем параметры кругов a, b и r в целые числа
    detected_circles_rounded = np.uint16(np.around(detected_circles))
    
    # Визуализируем результат на новом изображении
    if visualize:
        output_image_circles = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)

    centers = []
    # Рисуем обнаруженные круги
    if detected_circles_rounded is not None:
        for i in detected_circles_rounded[0, :]:
            center = (i[0], i[1])
            centers.append(center)

            if visualize:
                radius = i[2]
                # Рисуем внешний круг
                cv2.circle(output_image_circles, center, radius, (0, 255, 0), 2)
                # Рисуем центр круга
                cv2.circle(output_image_circles, center, 2, (0, 0, 255), 3)

    if visualize:
        plt.imshow(output_image_circles)
        plt.colorbar()
        plt.show()

    return centers


def substract_padding(centers: list, padding: Optional[int] = 10) -> list:
    """
    Корректирует координаты центров кругов, учитывая добавленную рамку.

    Аргументы:
    - centers: Список кортежей с координатами центров кругов.
    - padding: Ширина добавленной рамки. По умолчанию 10.

    Возвращает:
    - Список скорректированных координат центров кругов.
    """
    new_centers = []
    for c in centers:
        new_centers.append((c[0]-padding, c[1]-padding))
    return new_centers


def sort_centers(centers: list, img_path: str, visualize: Optional[bool] = False) -> list:
    """
    Сортирует центры кругов по их расстоянию до шестой точки и при необходимости визуализирует результат.

    Аргументы:
    - centers: Список кортежей с координатами центров кругов.
    - img_path: Путь к изображению.
    - visualize: Флаг, определяющий необходимость визуализации. По умолчанию False.

    Возвращает:
    - Список отсортированных координат центров кругов.
    """
    # Define the coordinates of the 6th point
    sixth_point = (40, 190)

    # Calculate distances from the sixth point to each of the centers
    distances = [np.sqrt((x - sixth_point[0])**2 + (y - sixth_point[1])**2) for x, y in centers]

    # Pair each center with its distance from the sixth point
    center_distances = list(zip(centers, distances))

    # Sort the centers by their distance from the sixth point
    sorted_centers = sorted(center_distances, key=lambda x: x[1])

    # Extract the sorted centers and their order
    sorted_centers_only = [center for center, distance in sorted_centers]

    if visualize:
        # Load the provided image
        img = Image.open(img_path)
        # Convert to RGB to plot color on top of the original image
        img_rgb = img.convert('RGB')
        draw = ImageDraw.Draw(img_rgb)

        # Draw the sixth point
        draw.ellipse((sixth_point[0]-3, sixth_point[1]-3, sixth_point[0]+3, sixth_point[1]+3), fill='blue', outline='blue')

        # Draw the centers, lines, and order
        for num, (x, y) in enumerate(sorted_centers_only):
            # Draw the center
            draw.ellipse((x-3, y-3, x+3, y+3), fill='red', outline='red')
            # Draw line from the center to the sixth point
            draw.line((x, y, sixth_point[0], sixth_point[1]), fill='green', width=1)
            # Annotate the order next to the center
            draw.text((x+5, y), f'{num+1}', fill='purple',)

        plt.imshow(img_rgb)
        plt.colorbar()
        plt.show()

    return sorted_centers_only


def define_corners(center: tuple, radius: Optional[int] = 15) -> tuple:
    """
    Определяет координаты углов квадрата вокруг центра круга с заданным радиусом.

    Аргументы:
    - center: Координаты центра круга.
    - radius: Радиус квадрата. По умолчанию 15.

    Возвращает:
    - Кортеж с координатами углов квадрата.
    """
    left = center[0] - radius
    right = center[0] + radius
    up = center[1] - radius
    down = center[1] + radius
    
    return (left, up, right, down)


def get_last_directory(path: str) -> str:
    """
    Возвращает имя последней директории в указанном пути.

    Аргументы:
    - path: Путь, из которого нужно извлечь имя последней директории.

    Возвращает:
    - Имя последней директории в пути.
    """
    directory_path = os.path.dirname(path)
    return os.path.basename(directory_path)



def crop_squares(image_path: str, squares: list, output_dir: str, visualize: Optional[bool] = False):
    """
    Вырезает квадраты из изображения по заданным координатам и сохраняет их.

    Аргументы:
    - image_path: Путь к изображению.
    - squares: Список кортежей, каждый из которых представляет координаты квадрата.
    - output_dir: Директория для сохранения вырезанных квадратов.
    - visualize: Флаг, определяющий необходимость визуализации. По умолчанию False.
    """

    subfoldername = os.path.basename(image_path).split('.')[0]
    foldername = get_last_directory(image_path)
    full_path = os.path.join(output_dir,foldername,subfoldername)
    os.makedirs(full_path, exist_ok=True)

    # Load the image
    img = Image.open(image_path)

    # Convert the image to RGB if it's not already
    if img.mode != 'RGB':
        img = img.convert('RGB')

    if visualize:
        draw = ImageDraw.Draw(img)

    # Draw each square
    for i, (left, up, right, down) in enumerate(squares):

        if visualize:
            # Draw a rectangle for each square
            draw.rectangle([(left, up), (right, down)], outline='yellow', width=2)

        crop = img.crop((left, up, right, down))
        crop_path = os.path.join(full_path, f"{i+1}.jpg")
        crop.save(crop_path)
    
    if visualize:
        plt.imshow(img)
        plt.colorbar()
        plt.show()

In [4]:
"""def combine_all_preprocessing_functions(image_path, output_dir, visualize=False):
    im = open_image_and_visualize(image_path, visualize=visualize)
    im = make_scalar_product_mask(im, visualize=visualize)
    im = make_binar(im, threshold=240)
    im = pad_borders(im, pad_width=10, visualize=visualize)
    im = smooth_circle_borders(im, visualize=visualize)
    centers = detect_circles(im, visualize=visualize)
    centers = substract_padding(centers, padding=10)
    centers = sort_centers(centers, image_path, visualize=visualize)
    squares = [define_corners(center, radius=20) for center in centers]
    crop_squares(image_path, squares, output_dir, visualize=visualize)
    return centers"""

'def combine_all_preprocessing_functions(image_path, output_dir, visualize=False):\n    im = open_image_and_visualize(image_path, visualize=visualize)\n    im = make_scalar_product_mask(im, visualize=visualize)\n    im = make_binar(im, threshold=240)\n    im = pad_borders(im, pad_width=10, visualize=visualize)\n    im = smooth_circle_borders(im, visualize=visualize)\n    centers = detect_circles(im, visualize=visualize)\n    centers = substract_padding(centers, padding=10)\n    centers = sort_centers(centers, image_path, visualize=visualize)\n    squares = [define_corners(center, radius=20) for center in centers]\n    crop_squares(image_path, squares, output_dir, visualize=visualize)\n    return centers'

In [5]:
def combine_functions_to_get_centers(image_path: str, visualize: Optional[bool] = False) -> list:
    """
    Комбинирует несколько функций для обработки изображения и получения центров кругов.

    Аргументы:
    - image_path: Путь к изображению.
    - visualize: Флаг, определяющий необходимость визуализации. По умолчанию False.

    Возвращает:
    - Список координат центров кругов.
    """
    im = open_image_and_visualize(image_path, visualize=visualize)
    im = make_scalar_product_mask(im, visualize=visualize)
    im = make_binar(im, threshold=240)
    im = pad_borders(im, pad_width=10, visualize=visualize)
    im = smooth_circle_borders(im, visualize=visualize)
    centers = detect_circles(im, visualize=visualize)
    centers = substract_padding(centers, padding=10)
    centers = sort_centers(centers, image_path, visualize=visualize)
    return centers

def save_squares(image_path: str, output_dir: str, centers: list, visualize: Optional[bool] = False):
    """
    Сохраняет квадраты, вырезанные вокруг центров кругов, в указанную директорию.

    Аргументы:
    - image_path: Путь к изображению.
    - output_dir: Директория для сохранения вырезанных квадратов.
    - centers: Список координат центров кругов.
    - visualize: Флаг, определяющий необходимость визуализации. По умолчанию False.
    """
    squares = [define_corners(center, radius=20) for center in centers]
    crop_squares(image_path, squares, output_dir, visualize=visualize)

## Predict image

In [6]:
print(8)

8


In [7]:
#! pip install ftfy regex tqdm
#! pip install git+https://github.com/openai/CLIP.git

In [8]:
def display_comparison(base_image: Any, compare_image: Any, similarity: float, i: int) -> None:
    """
    Отображает сравнение базового изображения с изображением для сравнения, включая показатель сходства.

    Аргументы:
    - base_image: Изображение для отображения как базовое.
    - compare_image: Изображение для сравнения с базовым.
    - similarity: Показатель сходства между изображениями.
    - i: Индекс изображения для сравнения.

    Возвращает:
    - None
    """
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(base_image)
    plt.title("Base Image")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(compare_image)
    plt.title(f"Compare to Image {i}\nSimilarity: {similarity.item():.4f}")
    plt.axis('off')

    plt.show()




## Preprocess Y

In [9]:
import easyocr

# Создание объекта reader
reader = easyocr.Reader(['en'], gpu=False,)  # Укажите 'gpu=True', если доступна GPU

# Функция для распознавания текста с изображения
def recognize_digit(image_path: str) -> str:
    """
    Распознает цифру на изображении с использованием EasyOCR.

    Аргументы:
    - image_path: Путь к изображению для распознавания.

    Возвращает:
    - Распознанную цифру в виде строки. В случае отсутствия распознавания возвращает "1".
    """
    result = reader.readtext(image_path, detail=1, paragraph=False, allowlist='12345')
    #print("result:", result)
    if not result:
        print("None is replaced with 1")
        return "1"
    for detection in result:
        bbox, text, confidence = detection
        #print(text, type(text), type(text[0])) 
        return text[0]

Using CPU. Note: This module is much faster with a GPU.


In [10]:
def save_bottom_half(input_folder: str, output_folder: str) -> None:
    """
    Вырезает и сохраняет нижнюю половину изображения 'y.jpg' из входной папки в выходную папку под именем 'im.jpg'.

    Аргументы:
    - input_folder: Папка, содержащая исходное изображение 'y.jpg'.
    - output_folder: Папка для сохранения результата.

    Возвращает:
    - None
    """
    # Open the input image
    input_image_path = os.path.join(input_folder, "y.jpg")
    output_image_path = os.path.join(output_folder, "im.jpg")

    with Image.open(input_image_path) as img:
        width, height = img.size
        
        # Calculate the box for the bottom half
        # The box is defined by (left, upper, right, lower) edges.
        box = (0, height // 2, width, height)
        
        # Crop the bottom half
        bottom_half = img.crop(box)
        
        # Save the cropped image to the specified output path
        bottom_half.save(output_image_path)

## Get metrics

In [11]:
import json


def find_matching_square(path_to_json: str) -> list:
    """
    Находит координаты прямоугольника, содержащего знак плюс, в JSON файле аннотаций.

    Аргументы:
    - path_to_json: Путь к JSON файлу с аннотациями.

    Возвращает:
    - Координаты прямоугольника, содержащего знак плюс, в формате списка из двух точек.
    """
    
    with open(path_to_json) as json_file:
        data = json.load(json_file)
    # First, let's find the coordinates of all pluses
    plus_signs = [shape['points'][0] for shape in data['shapes'] if shape['label'] == '+']

    # Now, we will check which rectangles contain these plus signs
    # A point is inside a rectangle if its x coordinate is between the x coordinates of the rectangle
    # and its y coordinate is between the y coordinates of the rectangle

    def is_point_in_rect(point, rect):
        # unpack points
        px, py = point
        (rx1, ry1), (rx2, ry2) = rect
        return rx1 <= px <= rx2 and ry1 <= py <= ry2

    # List to store rectangles that contain a plus
    rectangles_with_plus = []

    # Iterate over the shapes to find rectangles that contain a plus
    for shape in data['shapes']:
        if shape['shape_type'] == 'rectangle':
            # Get rectangle coordinates
            rect_coords = shape['points']
            # Check each plus sign
            for plus in plus_signs:
                if is_point_in_rect(plus, rect_coords):
                    return rect_coords

In [12]:
def find_point_and_rect(json_file_path: str) -> tuple:
    """
    Находит координаты точки и прямоугольника, соответствующие условиям задачи, основываясь на данных из JSON файла.

    Аргументы:
    - json_file_path: Путь к JSON файлу с аннотациями.

    Возвращает:
    - Кортеж, содержащий координаты точки, номер точки, координаты прямоугольника и смещение прямоугольника по оси X.
    """
    import json
    from math import sqrt

    # Загрузка данных из файла
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    
    # Извлечение специальной цифры из лейблов
    special_number = int(next(filter(str.isdigit, (shape['label'] for shape in data['shapes']))))

    ordinary_points = [shape for shape in data['shapes'] if shape['label'] == '+']
    icons = [shape for shape in data['shapes'] if shape['label'] == 'icon']
    point_a = (40, 190)  # Главная точка A
    
    # Вычисление условного расстояния и картиночного номера
    def conditional_distance(point_a, point_b):
        x_b_mod = point_b[0] % 200
        distance = sqrt((x_b_mod - point_a[0]) ** 2 + (point_b[1] - point_a[1]) ** 2)
        return distance

    ordinary_points_info = [{
        'coords': shape['points'][0],
        'distance': conditional_distance(point_a, shape['points'][0]),
        'picture_number': int(shape['points'][0][0] // 200) + 1
    } for shape in ordinary_points]

    ordinary_points_info.sort(key=lambda x: x['distance'])
    
    for index, point in enumerate(ordinary_points_info, start=1):
        point['length_number'] = index
    
    # Находим точку, у которой длинностный номер равен специальной цифре
    matching_point = next((p for p in ordinary_points_info if p['length_number'] == special_number), None)
    
    # Проверка и смещение прямоугольника
    def is_point_in_rect(point, rect):
        return rect[0][0] <= point[0] <= rect[1][0] and rect[0][1] <= point[1] <= rect[1][1]

    def find_shift_and_rect(point, icons):
        for icon in icons:
            for shift_x in range(0, 10001, 200):  # Предполагаем, что смещение может быть до 10000 пикселей
                shifted_rect = [[x + shift_x, y] for x, y in icon['points']]
                if is_point_in_rect(point, shifted_rect):
                    return shifted_rect, shift_x
        return None, None

    # Находим подходящий прямоугольник и требуемое смещение
    matching_rect, shift_x = find_shift_and_rect(matching_point['coords'], icons)

    return matching_point['coords'], matching_point['length_number'], matching_rect, shift_x


## Make benchmarked dataset

In [13]:
from PIL import Image

def save_correct_answer(rectangle_coords: list, image_path: str, cropped_image_path: str) -> None:
    """
    Вырезает и сохраняет указанный прямоугольник из изображения.

    Аргументы:
    - rectangle_coords: Координаты прямоугольника для вырезки.
    - image_path: Путь к исходному изображению.
    - cropped_image_path: Путь для сохранения вырезанного прямоугольника.

    Возвращает:
    - None
    """
    # Reload the original image
    image = Image.open(image_path)

    # Define the coordinates of the rectangle to be cropped
    #rectangle_coords = [[[77.76795580110496, 47.933701657458556], [123.20441988950277, 95.58011049723757]]]
    top_left_x, top_left_y = rectangle_coords[0]
    bottom_right_x, bottom_right_y = rectangle_coords[1]

    # Crop the image
    cropped_image = image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))

    # Save the cropped image to a file
    cropped_image.save(cropped_image_path)

In [31]:
input_image_path = "data/raw"
output_folder_path = "data/interim/orbits"

files = os.listdir(input_image_path)
jpg_files = [file for file in files if file.endswith('.jpg')]
len(jpg_files)

for file in tqdm(jpg_files):
    foldername = split_and_save_image(file, input_image_path, output_folder_path, save_to_tmp=False)

100%|██████████| 970/970 [00:16<00:00, 60.14it/s]


In [15]:
input_image_path = "data/raw"
benchmarked_path = "data/benchmarked"
parsed_path = "data/interim/orbits"


os.makedirs(benchmarked_path, exist_ok=True)

files = os.listdir(input_image_path)
json_files = [file for file in files if file.endswith('.json')]
len(json_files)

for file in tqdm(json_files):
    rectangle_coords = find_matching_square(os.path.join(input_image_path, file))
    #print(square)
    foldername = file.split(".")[0]
    os.makedirs(os.path.join(benchmarked_path, foldername), exist_ok=True)
    try:
        save_correct_answer(rectangle_coords, 
                            image_path=os.path.join(input_image_path, f"{foldername}.jpg"), 
                            cropped_image_path=os.path.join(benchmarked_path, foldername, "correct.jpg"))

        save_bottom_half(input_folder=os.path.join(parsed_path, foldername),
                        output_folder=os.path.join(benchmarked_path, foldername))
    except:
        print(foldername)

                                 
#for file in tqdm(jpg_files[:5]):

 18%|█▊        | 176/970 [00:02<00:09, 83.22it/s]

183730


 23%|██▎       | 221/970 [00:02<00:07, 98.11it/s]

183887


 81%|████████▏ | 789/970 [00:08<00:01, 109.04it/s]

...

 96%|█████████▋| 936/970 [00:09<00:00, 129.95it/s]

186462
186464
186467
186492
186494
186497
186519
186522
186524
186525


 98%|█████████▊| 951/970 [00:09<00:00, 99.70it/s] 

186585
186596
186616
186628
186637
186639
186658


100%|██████████| 970/970 [00:10<00:00, 96.86it/s] 


## Pipeline

In [18]:
import os
from tqdm import tqdm

In [19]:
def center_for_answer(centers: list, idx: int, digit: int) -> list:
    """
    Рассчитывает центр для ответа на основе индекса и распознанной цифры.

    Аргументы:
    - centers: Список центров.
    - idx: Индекс выбранного изображения.
    - digit: Распознанная цифра.

    Возвращает:
    - Центр для ответа в виде списка координат.
    """
    width = 200
    center = list(centers[digit-1])
    center[0] = center[0] + (idx-1) * width
    return center


def is_point_in_rectangle(point: tuple, rectangle: tuple) -> bool:
    """
    Проверяет, находится ли точка внутри заданного прямоугольника.

    Аргументы:
    - point: Координаты точки (x, y).
    - rectangle: Координаты прямоугольника ((x1, y1), (x2, y2)).

    Возвращает:
    - True, если точка находится внутри прямоугольника, иначе False.
    """
    x, y = point
    x1, y1 = rectangle[0]
    x2, y2 = rectangle[1]
    
    if (x >= x1 and x <= x2) and (y >= y1 and y <= y2):
        return True
    else:
        return False

In [32]:
input_image_path = "data/raw"
output_folder_path = "data/interim/orbits"
output_for_objects_path = "data/interim/objects"

files = os.listdir(input_image_path)
jpg_files = [file for file in files if file.endswith('.jpg')]
#len(jpg_files)

for file in tqdm(jpg_files[4:7]):
    try:
        foldername = split_and_save_image(file, input_image_path, output_folder_path, save_to_tmp=True)
        name_without_format = file.split(".")[0]
        digit = recognize_digit(os.path.join(output_folder_path, foldername, "y.jpg"))
        #print(file, digit)
        save_bottom_half(input_folder=os.path.join(output_folder_path, foldername),
                        output_folder=os.path.join(output_for_objects_path, foldername))
        list_of_centers = []
        for i in range(1, 6):
            ##print("AAAAAA: ", i, file)
            image_path = os.path.join(output_folder_path, foldername, f"x_{i}.jpg")
            centers = combine_all_preprocessing_functions(image_path, 
                                                        output_for_objects_path, 
                                                        visualize=False)
            list_of_centers.append(centers)

        idx = compare_images(base_image_folder=os.path.join(output_for_objects_path, foldername), 
                            compare_image_folder=os.path.join(output_for_objects_path, foldername), 
                            digit=digit, 
                            model=model, 
                            preprocess=preprocess,
                            visualize=False,)
        centers = list_of_centers[idx-1]
        matching_point, number, correct_square, shift_x = find_point_and_rect(json_file_path=os.path.join(input_image_path, f"{name_without_format}.json"))
        answer =  center_for_answer(centers, idx, int(digit))
        #print(answer)
        print(file)
        print(answer, correct_square, is_point_in_rectangle(point=answer, rectangle=correct_square))
    #print(idx, answer, correct_square)
    #print("All centers:", centers)
        #draw_squares(image_path, squares, output_dir, visualize=False, save_to_tmp=True)
    except:
        print("SOME ERROR:", file)

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:00<00:00,  3.68it/s]

None is replaced with 1
SOME ERROR: 183121.jpg


 67%|██████▋   | 2/3 [00:00<00:00,  4.05it/s]

SOME ERROR: 183122.jpg


100%|██████████| 3/3 [00:00<00:00,  4.06it/s]

SOME ERROR: 183125.jpg


## Test samples

In [22]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

   ---------------------------------------- 0.0/53.4 kB ? eta -:--:--
   ----------------------- ---------------- 30.7/53.4 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 53.4/53.4 kB 685.0 kB/s eta 0:00:00
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.6
    Uninstalling wcwidth-0.2.6:
      Successfully uninstalled wcwidth-0.2.6



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\epsil\AppData\Local\Temp\pip-req-build-aovygaet'

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Cloning https://github.com/openai/CLIP.git to c:\users\epsil\appdata\local\temp\pip-req-build-aovygaet
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369580 sha256=3486c1dc136bb5e7e882c666ef21836c59a96108f50aa4817cb4142d9bcd856e
  Stored in directory: C:\Users\epsil\AppData\Local\Temp\pip-ephem-wheel-cache-8a2t5hx4\wheels\3f\7c\a4\9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [20]:
import torch
import numpy as np
import clip
import torch
from torch import nn
from PIL import Image

# Загрузка предварительно обученной модели CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

class CLIPFineTuneModel(nn.Module):
    """
    Класс для настройки модели CLIP с добавлением линейного слоя для финетюнинга.

    Атрибуты:
    - clip_model (torch.nn.Module): предобученная модель CLIP.
    - linear (torch.nn.Linear): линейный слой для трансформации признаков.
    - cosine_similarity (torch.nn.CosineSimilarity): слой для вычисления косинусного сходства между векторами признаков.

    Методы:
    - forward(img1, img2): Проход вперед, который принимает пары изображений и возвращает оценку их сходства.
    """

    def __init__(self, clip_model):
        """
        Инициализирует класс с предобученной моделью CLIP и добавляет линейный слой.

        Параметры:
        - clip_model (torch.nn.Module): предобученная модель CLIP.
        """
        super(CLIPFineTuneModel, self).__init__()
        self.clip_model = clip_model
        self.linear = nn.Linear(512, 64)
        self.cosine_similarity = nn.CosineSimilarity(dim=1)

    def forward(self, img1, img2):
        """
        Выполняет проход вперед, принимая два изображения и вычисляя их сходство.

        Параметры:
        - img1 (torch.Tensor): тензор первого изображения.
        - img2 (torch.Tensor): тензор второго изображения.

        Выходные данные:
        - similarity (torch.Tensor): тензор с оценками сходства изображений.
        """
        with torch.no_grad():
            features1 = self.clip_model.encode_image(img1).to(dtype=torch.float32)
            features2 = self.clip_model.encode_image(img2).to(dtype=torch.float32)

        transformed_features1 = self.linear(features1).to(dtype=torch.float32)
        transformed_features2 = self.linear(features2).to(dtype=torch.float32)
        transformed_features1 = transformed_features1 / transformed_features1.norm(dim=-1, keepdim=True)
        transformed_features2 = transformed_features2 / transformed_features2.norm(dim=-1, keepdim=True)

        similarity = self.cosine_similarity(transformed_features1, transformed_features2).unsqueeze(-1).to(dtype=torch.float32)
        return similarity

c:\Users\epsil\AppData\Local\Programs\Python\Python311\Lib\site-packages\clip\clip.py:57: UserWarning: C:\Users\epsil/.cache/clip\ViT-B-32.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(f"{download_target} exists, but the SHA256 checksum does not match; re-downloading the file")
100%|████████████████████████████████████████| 338M/338M [07:45<00:00, 761kiB/s]


In [21]:
# Инициализация модели надстройки
finetune_model = CLIPFineTuneModel(model).to(device)

# Путь к сохраненной модели
model_path = 'finetuned_clip_model_roc093.pth'

# Загрузка весов в модель
#finetune_model.load_state_dict(torch.load(model_path))
finetune_model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
finetune_model.eval()  # Переключение модели в режим оценки

CLIPFineTuneModel(
  (clip_model): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
      (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): Sequential(
          (0): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
            )
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=768, out_features=3072, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=3072, out_features=768, bias=True)
            )
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          )
          (1): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicall

In [22]:
"""
def get_image_features(image_path, model, preprocess):
    image = Image.open(image_path).convert("RGB")
    image_processed = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model.encode_image(image_processed).to(dtype=torch.float32)
    return features, image
"""
import os

# Сравнение изображений и вывод результатов
"""def compare_images(base_image_folder, compare_image_folder, digit, finetune_model, preprocess, visualize=False):
    base_image_path = os.path.join(base_image_folder, "im.jpg")
    base_features, base_image = get_image_features(base_image_path, model, preprocess)
    
    max_similarity = -1  # Инициализируем переменную для сохранения наибольшей схожести
    max_similarity_index = -1  # Инициализируем переменную для сохранения индекса наибольшей схожести

    for i in range(1, 6):
        compare_image_path = os.path.join(compare_image_folder, f"x_{i}", f"{digit}.jpg")
        try:
            #print("PATH: ", compare_image_path)
            compare_features, compare_image = get_image_features(compare_image_path, model, preprocess)
            similarity = (base_features @ compare_features.T).cpu().numpy()
            if similarity > max_similarity:
                max_similarity = similarity
                max_similarity_index = i
            if visualize:
                display_comparison(base_image, compare_image, similarity, i)
        except FileNotFoundError:
            print(f"Файл {compare_image_path} не найден.")

    return max_similarity_index  # Возвращает индекс с наибольшей схожестью"""



def compare_images(base_image_folder: str, compare_image_folder: str, digit: int, finetune_model, preprocess, visualize: bool = False) -> int:
    """
    Функция для сравнения изображений и определения наиболее похожего.
    
    :param base_image_folder: Путь к папке с базовым изображением.
    :param compare_image_folder: Путь к папке с изображениями для сравнения.
    :param digit: Цифра для определения номера сравниваемого изображения.
    :param finetune_model: Модель для сравнения изображений.
    :param preprocess: Функция предварительной обработки изображений.
    :param visualize: Флаг для визуализации процесса сравнения (по умолчанию False).
    
    :return: Индекс наиболее похожего изображения.
    """
    base_image_path = os.path.join(base_image_folder, "im.jpg")
    #base_features, base_image = get_image_features(base_image_path, clip_model, preprocess)
    base_image_raw = Image.open(base_image_path).convert("RGB")
    base_image = preprocess(base_image_raw).unsqueeze(0).to(device)

    max_similarity = -1
    max_similarity_index = -1

    for i in range(1, 6):
        compare_image_path = os.path.join(compare_image_folder, f"x_{i}", f"{digit}.jpg")
        compare_image_raw = Image.open(compare_image_path).convert("RGB")
        compare_image = preprocess(compare_image_raw).unsqueeze(0).to(device)
        try:
            #compare_features, compare_image = get_image_features(compare_image_path, clip_model, preprocess)
            # Использование модели надстройки для сравнения
            similarity = finetune_model(base_image, compare_image) #base_features, compare_features).cpu().numpy()
            if similarity > max_similarity:
                max_similarity = similarity
                max_similarity_index = i
            if visualize:
                display_comparison(base_image_raw, compare_image_raw, similarity, i)
        except FileNotFoundError:
            print(f"Файл {compare_image_path} не найден.")

    return max_similarity_index

In [23]:
def median(values: List[float]) -> Optional[float]:
    """
    Вычисляет медиану списка значений.

    Аргументы:
    - values: Список значений.

    Возвращает:
    - Медианное значение списка. Возвращает None, если список пуст.
    """
    sorted_values = sorted(values)
    n = len(sorted_values)
    if n == 0:
        return None
    if n % 2 == 1:
        return sorted_values[n // 2]
    else:
        return (sorted_values[n // 2 - 1] + sorted_values[n // 2]) / 2

def calculate_median_coordinates(list_of_lists: List[List[tuple]]) -> List[tuple]:
    """
    Вычисляет медианные координаты для списка списков координат.

    Аргументы:
    - list_of_lists: Список списков координат.

    Возвращает:
    - Список медианных координат.
    """
    median_coordinates = []
    for i in range(5):  # Предполагаем, что требуется 5 пар координат
        x_values = []
        y_values = []
        for lst in list_of_lists:
            if i < len(lst):  # Убедимся, что индекс не выходит за пределы списка
                x_values.append(lst[i][0])
                y_values.append(lst[i][1])
        median_x = median(x_values)
        median_y = median(y_values)
        if median_x and median_y:
            median_coordinates.append((median_x, median_y))
    return median_coordinates

In [29]:
def calculate_metrics(input_image_path: str = "data/raw",
                      output_folder_path: str = "data/interim/orbits",
                      output_for_objects_path: str = "data/interim/objects",
                      path_to_chosen_files: Optional[str] = None) -> None:
    """
    Вычисляет метрики для набора изображений.

    Аргументы:
    - input_image_path: Путь к входным изображениям.
    - output_folder_path: Путь к промежуточным результатам.
    - output_for_objects_path: Путь к обработанным объектам.
    - path_to_chosen_files: Путь к файлу со списком выбранных файлов для обработки.

    Возвращает:
    - None
    """

    is_correct = []
    files = os.listdir(input_image_path)

    if path_to_chosen_files:
        with open(path_to_chosen_files, 'r') as file:
            valid_names = file.read().split("\n")
            jpg_files = [file for file in files if file.split('.')[0] in valid_names and file.endswith('.jpg')]
            print(valid_names)
    else:
        jpg_files = [file for file in files if file.endswith('.jpg')]


    for file in tqdm(jpg_files[:10]):
    #for file in tqdm(errors[:3]):
        #try:
        foldername = split_and_save_image(file, input_image_path, output_folder_path, save_to_tmp=True)
        name_without_format = file.split(".")[0]
        digit = recognize_digit(os.path.join(output_folder_path, foldername, "y.jpg"))
        save_bottom_half(input_folder=os.path.join(output_folder_path, foldername),
                        output_folder=os.path.join(output_for_objects_path, foldername))
        list_of_centers = []
        for i in range(1, 6):
            image_path = os.path.join(output_folder_path, foldername, f"x_{i}.jpg")
            centers = combine_functions_to_get_centers(image_path, visualize=False)
            list_of_centers.append(centers)

        centers = calculate_median_coordinates(list_of_centers)
        for i in range(1, 6):
            image_path = os.path.join(output_folder_path, foldername, f"x_{i}.jpg")
            save_squares(image_path, output_for_objects_path, centers, visualize=True)
        
        """for i in range(1, 6):
            image_path = os.path.join(output_folder_path, foldername, f"x_{i}.jpg")
            centers = combine_all_preprocessing_functions(image_path, 
                                                        output_for_objects_path, 
                                                        visualize=False)
            list_of_centers.append(centers)"""
        

        idx = compare_images(base_image_folder=os.path.join(output_for_objects_path, foldername), 
                            compare_image_folder=os.path.join(output_for_objects_path, foldername), 
                            digit=digit, 
                            finetune_model=finetune_model, 
                            preprocess=preprocess,
                            visualize=True)        
        #centers = calculate_median_coordinates(list_of_centers)

       
        
        print("LIST OF CENTRERS:", list_of_centers)
        print("CENTERS: ", centers)
        print("IDX:", idx, "DIGIT:", digit)
        matching_point, number, correct_square, shift_x = find_point_and_rect(json_file_path=os.path.join(input_image_path, f"{name_without_format}.json"))
        answer =  center_for_answer(centers, idx, int(digit))
        print("ANSWER:", answer, "correct_square:", correct_square, "shift_x:", shift_x)
        is_ok = int(is_point_in_rectangle(point=answer, rectangle=correct_square))
        
        if not is_ok:
            print(file)
        is_correct.append(is_ok)
        #print(file)
        #print(answer, correct_square, is_point_in_rectangle(point=answer, rectangle=correct_square))
    #except:
        #is_correct.append(0)
        #print("SOME ERROR:", file)
        
    print(f"Number of evaluated samples: {len(is_correct)}")
    print(f"Accuracy: {sum(is_correct)/len(is_correct)}")
    print(is_correct)

In [33]:
calculate_metrics(path_to_chosen_files="test_files.txt")

['185744', '183271', '184790', '183166', '183965', '184281', '186433', '183131', '183415', '184230', '185242', '186498', '183751', '183474', '183487', '185946', '185003', '183959', '183610', '185771', '183255', '186137', '185245', '183629', '184824', '183641', '183257', '185681', '185894', '185630', '185658', '185141', '184373', '185453', '183235', '183135', '183770', '186414', '184537', '186446', '184640', '183419', '184822', '185492', '186428', '184148', '183225', '183535', '184208', '186507', '184106', '186028', '185524', '184619', '183218', '183187', '185232', '185097', '185596', '185346', '185014', '184183', '185182', '185291', '185536', '186372', '184018', '185896', '184935', '183396', '183660', '185607', '183988', '184236', '185851', '184987', '185352', '185407', '185364', '186350', '184605', '184849', '186377', '185588', '185556', '185717', '186064', '185384', '184993', '185069', '185020', '184921', '']


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'data/interim/objects\\tmp\\im.jpg'

In [ ]:
calculate_metrics(path_to_chosen_files="test_files.txt")

In [70]:
errors = ["183135.jpg", "183225.jpg", "183255.jpg", "183257.jpg", "183415.jpg", "183474.jpg", "183487.jpg"]

In [29]:
jpg_files[:5]

['183099.jpg', '183103.jpg', '183108.jpg', '183118.jpg', '183121.jpg']

In [28]:
# Define the paths
input_image_path = 'orbits/train'
output_folder_path = 'orbits/parsed_input'

#
filenames = ["183199.jpg", "183201.jpg", "183203.jpg", "183205.jpg", 
"183208.jpg", "183218.jpg", "183220.jpg"]
filenames = ["183125.jpg", "183108.jpg", "183239.jpg"]

# Call the function
for filename in tqdm(filenames):
    split_and_save_image(filename, input_image_path, output_folder_path, save_to_tmp=True)

  0%|          | 0/3 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'orbits/train\\183125.jpg'